# How much do Covariates Matter?

## Motivation

In regression analyses, we often wonder about "how much covariates matter?" for explaining the relationship between a target variable $D$ and an outcome variable $Y$. 

For example, we might start analysing the gender wage gap with a simple regression model as `log(wage) on gender`. But arguably, men and women differ in many socio-economic characteristics: they might have different (average) levels of education or career experience, and they might work in different industries and select into different higher- or lower-paying industries. So which fraction of the gender wage gap can be explained by these observable characteristics? 

In this notebook, we will compute and decompose the gender wage gab based on a subset of the PSID data set using a method commonly known as the 
"Gelbach" Decomposition ([Gelbach, JoLE 2016](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1425737)). 

We start with loading a subset of the PSID data provided by the AER R package. 

In [1]:
import re

import pandas as pd

import pyfixest as pf

psid = pd.read_csv(
    "https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/refs/heads/master/csv/AER/PSID7682.csv"
)
psid["experience"] = pd.Categorical(psid["experience"])
psid["year"] = pd.Categorical(psid["year"])
psid.head()

,rownames,experience,weeks,occupation,industry,south,smsa,married,gender,union,education,ethnicity,wage,year,id
0,1,3,32,white,no,yes,no,yes,male,no,9,other,260,1976,1
1,2,4,43,white,no,yes,no,yes,male,no,9,other,305,1977,1
2,3,5,40,white,no,yes,no,yes,male,no,9,other,402,1978,1
3,4,6,39,white,no,yes,no,yes,male,no,9,other,402,1979,1
4,5,7,42,white,yes,yes,no,yes,male,no,9,other,429,1980,1


Computing a first correlation between gender and wage, we find that males earn on average 0.474 log points
more than women. 

In [2]:
fit_base = pf.feols("log(wage) ~ gender", data=psid, vcov="hetero")
fit_base.summary()

###

Estimation:  OLS
Dep. var.: log(wage), Fixed effects: 0
Inference:  hetero
Observations:  4165

| Coefficient    |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:---------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| Intercept      |      6.255 |        0.020 |   320.714 |      0.000 |  6.217 |   6.294 |
| gender[T.male] |      0.474 |        0.021 |    22.818 |      0.000 |  0.434 |   0.515 |
---
RMSE: 0.436 R2: 0.106 


To examine the impact of observable on the relationship between wage and gender, a common strategy in applied research is to incrementally add a set of covariates to the baseline regression model of `log(wage) on gender`. Here, we will incrementally add the following covariates:  

- education,
- experience
- occupation, 
- industry 
- year 
- ethnicity

We can do so by using **multiple estimation syntax**: 

In [3]:
fit_stepwise1 = pf.feols(
    "log(wage) ~ gender + csw0(education, experience, occupation, industry, year, ethnicity)",
    data=psid,
)
pf.etable(fit_stepwise1)

GT(_tbl_data=   level_0             level_1                      0                      1  \
0     coef      gender[T.male]  0.474*** <br> (0.021)  0.474*** <br> (0.019)   
1     coef           education                         0.065*** <br> (0.002)   
2     coef     experience[T.2]                                                 
3     coef     experience[T.3]                                                 
4     coef     experience[T.4]                                                 
..     ...                 ...                    ...                    ...   
60    coef  ethnicity[T.other]                                                 
61    coef           Intercept  6.255*** <br> (0.020)  5.419*** <br> (0.034)   
62   stats        Observations                   4165                   4165   
63   stats           S.E. type                    iid                    iid   
64   stats       R<sup>2</sup>                  0.106                  0.260   

                        2                      3                      4  \
0   0.425*** <br> (0.018)  0.444*** <br> (0.018)  0.428*** <br> (0.018)   
1   0.075*** <br> (0.002)  0.061*** <br> (0.003)  0.063*** <br> (0.003)   
2      0.147 <br> (0.156)     0.156 <br> (0.155)     0.158 <br> (0.154)   
3      0.273 <br> (0.139)    0.284* <br> (0.138)    0.278* <br> (0.138)   
4    0.377** <br> (0.137)   0.389** <br> (0.136)   0.385** <br> (0.135)   
..                    ...                    ...                    ...   
60                                                                        
61  4.566*** <br> (0.134)  4.664*** <br> (0.133)  4.636*** <br> (0.133)   
62                   4165                   4165                   4165   
63                    iid                    iid                    iid   
64                  0.376                  0.387                  0.391   

                        5                      6  
0   0.432*** <br> (0.016)  0.410*** <br> (0.017)  
1   0.060*** <br> (0.002)  0.059*** <br> (0.002)  
2      0.124 <br> (0.137)     0.128 <br> (0.136)  
3      0.232 <br> (0.122)    0.241* <br> (0.122)  
4     0.287* <br> (0.121)    0.298* <br> (0.120)  
..                    ...                    ...  
60                         0.133*** <br> (0.020)  
61  4.672*** <br> (0.118)  4.570*** <br> (0.118)  
62                   4165                   4165  
63                    iid                    iid  
64                  0.520                  0.525  

[65 rows x 9 columns], _body=<great_tables._gt_data.Body object at 0x000001C8A4ACAD80>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='(7)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001C8A492E060>, _spanners=Spanners([SpannerInfo(spanner_id='log(wage)', spanner_level=1, spanner_label='log(wage)', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2', '3', '4', '5', '6'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_n

Because the table is so long that it's hard to see anything, we restrict it to display only a few variables: 

In [4]:
pf.etable(fit_stepwise1, keep=["gender", "ethnicity", "education"])

GT(_tbl_data=  level_0             level_1                      0                      1  \
0    coef      gender[T.male]  0.474*** <br> (0.021)  0.474*** <br> (0.019)   
1    coef  ethnicity[T.other]                                                 
2    coef           education                         0.065*** <br> (0.002)   
3   stats        Observations                   4165                   4165   
4   stats           S.E. type                    iid                    iid   
5   stats       R<sup>2</sup>                  0.106                  0.260   

                       2                      3                      4  \
0  0.425*** <br> (0.018)  0.444*** <br> (0.018)  0.428*** <br> (0.018)   
1                                                                        
2  0.075*** <br> (0.002)  0.061*** <br> (0.003)  0.063*** <br> (0.003)   
3                   4165                   4165                   4165   
4                    iid                    iid                    iid   
5                  0.376                  0.387                  0.391   

                       5                      6  
0  0.432*** <br> (0.016)  0.410*** <br> (0.017)  
1                         0.133*** <br> (0.020)  
2  0.060*** <br> (0.002)  0.059*** <br> (0.002)  
3                   4165                   4165  
4                    iid                    iid  
5                  0.520                  0.525  , _body=<great_tables._gt_data.Body object at 0x000001C8A4ABE6C0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='(7)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001C8A4A271A0>, _spanners=Spanners([SpannerInfo(spanner_id='log(wage)', spanner_level=1, spanner_label='log(wage)', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2', '3', '4', '5', '6'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001C8A4A27B00>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', '

We see that the coefficient on gender is roughly the same in all models. Tentatively, we might already conclude that the observable characteristics in the data do not explain a large part of the gender wage gap. 

But how much do differences in education matter? We have computed 6 additional models that contain education as a covariate. The obtained point estimates 
vary between $0.059$ and $0.075$. Which of these numbers should we report?  

Additionally, note that while we have only computed 6 additional models with covariates, the number of possible models is much larger. 
If I did the math correctly, simply by additively and incrementally adding covariates, we could have computed $57$ different models (not all of which would have included `education` as a control).

As it turns out, different models **will lead to different point estimates**. The order of incrementally adding covariates **might** impact our conclusion. To illustrate this, we keep the same ordering as before, but start with `ethnicity` as our first variable: 

In [5]:
fit_stepwise2 = pf.feols(
    "log(wage) ~ gender + csw0(ethnicity, education, experience, occupation, industry, year)",
    data=psid,
)
pf.etable(fit_stepwise2, keep=["gender", "ethnicity", "education"])

GT(_tbl_data=  level_0             level_1                      0                      1  \
0    coef      gender[T.male]  0.474*** <br> (0.021)  0.436*** <br> (0.022)   
1    coef  ethnicity[T.other]                         0.227*** <br> (0.026)   
2    coef           education                                                 
3   stats        Observations                   4165                   4165   
4   stats           S.E. type                    iid                    iid   
5   stats       R<sup>2</sup>                  0.106                  0.121   

                       2                      3                      4  \
0  0.450*** <br> (0.020)  0.399*** <br> (0.019)  0.418*** <br> (0.019)   
1  0.141*** <br> (0.024)  0.158*** <br> (0.023)  0.151*** <br> (0.022)   
2  0.064*** <br> (0.002)  0.074*** <br> (0.002)  0.060*** <br> (0.003)   
3                   4165                   4165                   4165   
4                    iid                    iid                    iid   
5                  0.266                  0.383                  0.394   

                       5                      6  
0  0.404*** <br> (0.019)  0.410*** <br> (0.017)  
1  0.146*** <br> (0.022)  0.133*** <br> (0.020)  
2  0.062*** <br> (0.003)  0.059*** <br> (0.002)  
3                   4165                   4165  
4                    iid                    iid  
5                  0.397                  0.525  , _body=<great_tables._gt_data.Body object at 0x000001C8A492D0A0>, _boxhead=Boxhead([ColInfo(var='level_0', type=<ColInfoTypeEnum.row_group: 3>, column_label='level_0', column_align='center', column_width=None), ColInfo(var='level_1', type=<ColInfoTypeEnum.stub: 2>, column_label='level_1', column_align='center', column_width=None), ColInfo(var='0', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='(4)', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='(5)', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='(6)', column_align='center', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='(7)', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001C8A49881A0>, _spanners=Spanners([SpannerInfo(spanner_id='log(wage)', spanner_level=1, spanner_label='log(wage)', spanner_units=None, spanner_pattern=None, vars=['0', '1', '2', '3', '4', '5', '6'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001C8A49DBD70>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', '

We obtain 5 new coefficients on `education` that vary between 0.074 and 0.059. 

So, which share of the "raw" gender wage gap can be attributed to differences in education between men and women? Should we report 0.075? Or 
0.059? Which value should we pick? 

To help us with this problem, Gelbach (2016, JoLE) develops a decomposition procedure building on the omitted variable bias formula that produces a single value for the contribution of a given covariate, say education, to the gender wage gap.

## `PyFixest` Example

To employ Gelbach's decomposition in `pyfixest`, we start with the **full** regression model that contains **all variables of interest**: 

In [6]:
fit_full = pf.feols(
    "log(wage) ~ gender + ethnicity + education + experience + occupation + industry +year",
    data=psid,
)

After fitting the **full model**, we can run the decomposition procedure by calling the `decompose()` method. The only required argument is to specify the target parameter, which in this case is "gender". Inference is conducted via a non-parametric bootstrap and can optionally be turned off.

In [7]:
gb = fit_full.decompose(param="gender[T.male]", digits=5)

100%|██████████| 1000/1000 [00:28<00:00, 35.39it/s]


As before, this produces a pretty big output table that reports 
- the **direct effect** of the regression of `log(wage) ~ gender`
- the **full effect** of gender on log wage using the **full regression** with all control variables
- the **explained effect** as the difference between the full and direct effect
- a **single scalar value** for the individual contributions of a covariate to overall **explained effect** 

For our example at hand, the additional covariates only explain a tiny fraction of the differences in log wages between men and women - 0.064 points. 
Of these, around one third can be attributed to ethnicity, 0.00064 to years of eduaction, etc. 

In [8]:
pf.make_table(gb)

,direct_effect,full_effect,explained_effect
gender[T.male],0.47447,0.41034,0.06413
,"[0.46299, 0.48089]","[0.38583, 0.41830]","[0.06260, 0.07716]"
ethnicity[T.other],,,0.02275
,,,"[0.01721, 0.02713]"
education,,,0.00064
,,,"[-0.00702, -0.00450]"
experience[T.2],,,-0.00030
,,,"[-0.00059, 0.00001]"
experience[T.3],,,-0.00234
,,,"[-0.00104, -0.00030]"


Because experience is a discrete variable, the table gets pretty unhandy: we produce one estimate for "each" level. Luckily, Gelbach's decomposition 
allows us to group individual contributions into a single number. In the `decompose()` method, we can combine variables via the `combine_covariates` argument: 

In [9]:
gb2 = fit_full.decompose(
    param="gender[T.male]",
    combine_covariates={
        "experience": re.compile("experience"),
        "occupation": re.compile("occupation"),
        "industry": re.compile("industry"),
        "year": re.compile("year"),
        "ethnicity": re.compile("ethnicity"),
    },
)

100%|██████████| 1000/1000 [00:12<00:00, 79.92it/s]


We now report a single value for "experience", which explains a good chunk - around two thirds - of the gender wage gap. 

In [10]:
pf.make_table(gb2)

,direct_effect,full_effect,explained_effect
gender[T.male],0.4745,0.4103,0.0635
,"[0.4483, 0.4985]","[0.4124, 0.4211]","[0.0466, 0.0703]"
experience,,,0.0390
,,,"[0.0390, 0.0426]"
occupation,,,-0.0165
,,,"[-0.0152, -0.0135]"
industry,,,0.0182
,,,"[0.0159, 0.0183]"
year,,,0.0000
,,,"[-0.0166, 0.0022]"


We can aggregate even more to "individual level" and "job" level variables: 

In [11]:
gb3 = fit_full.decompose(
    param="gender[T.male]",
    combine_covariates={
        "job": re.compile(r".*(occupation|industry).*"),
        "personal": re.compile(r".*(education|experience|ethnicity).*"),
        "year": re.compile("year"),
    },
)

100%|██████████| 1000/1000 [00:10<00:00, 91.37it/s]


In [12]:
pf.make_table(gb3)

GT(_tbl_data=            index     direct_effect       full_effect    explained_effect
0  gender[T.male]            0.4745            0.4103              0.0641
1                  [0.4656, 0.4796]  [0.4184, 0.4233]    [0.0472, 0.0563]
2             job                                                  0.0017
3                                                      [-0.0054, -0.0006]
4        personal                                                  0.0624
5                                                        [0.0458, 0.0556]
6            year                                                  0.0000
7                                                       [-0.0030, 0.0111], _body=<great_tables._gt_data.Body object at 0x000001C8A488ABA0>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='center', column_width=None), ColInfo(var='direct_effect', type=<ColInfoTypeEnum.default: 1>, column_label='direct_effect', column_align='center', column_width=None), ColInfo(var='full_effect', type=<ColInfoTypeEnum.default: 1>, column_label='full_effect', column_align='center', column_width=None), ColInfo(var='explained_effect', type=<ColInfoTypeEnum.default: 1>, column_label='explained_effect', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001C8A48892E0>, _spanners=Spanners([]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=[''], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001C8A48885C0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, cat

## Literature 

- ["When do Covariates Matter? And Which Ones, and How Much?" by Gelbach, Jonah B. (2016), Journal of Labor Economics](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1425737)